In [1]:
import math
import pandas as pd

class TwoWaySlab:
    def __init__(self, L, S, Ln, Sn, o_dl, o_ll, bar_dia, fc, fy, ll_red, wc, sw, c_case, include_sw):
        self.L = L
        self.S = S
        self.Ln = Ln /1000
        self.Sn = Sn /1000
        self.t = self.thickness(L, S, fc, fy, wc)
        self.o_dl = self.total_dl(o_dl, self.t, sw, include_sw)
        self.o_ll = self.live_load_red(L, S, Ln, Sn, o_ll, ll_red)
        self.bar_dia = bar_dia
        self.da = self.effective_depth_a(self.t, bar_dia)
        self.db = self.effective_depth_b(self.t, bar_dia)
        self.ult_load = self.ultimate_load(self.o_dl, self.o_ll)
        self.bar_area = self.bar_area(bar_dia)
        
        self.ca_neg_df = self.load_dataframe('ca-neg.csv')
        self.ca_posll_df = self.load_dataframe('ca-posll.csv')
        self.ca_posdl_df = self.load_dataframe('ca-posdl.csv')
        self.cb_neg_df = self.load_dataframe('cb-neg.csv')
        self.cb_posll_df = self.load_dataframe('cb-posll.csv')
        self.cb_posdl_df = self.load_dataframe('cb-posdl.csv')
        
        self.aci_coefficients = self.calculate_aci_coefficients(Ln, Sn, c_case)
        
        self.Ma_neg = self.Moment_negative(Sn, self.ult_load, self.aci_coefficients['ca_neg'])
        self.Ma_pos = self.Moment_positive(Sn, self.o_dl, self.o_ll, self.aci_coefficients['ca_posll'], self.aci_coefficients['ca_posdl'])
        self.Mb_neg = self.Moment_negative(Ln, self.ult_load, self.aci_coefficients['cb_neg'])
        self.Mb_pos = self.Moment_positive(Ln, self.o_dl, self.o_ll, self.aci_coefficients['cb_posll'], self.aci_coefficients['cb_posdl'])
        self.Ma_sup = self.Moment_support(self.Ma_neg)
        self.Mb_sup = self.Moment_support(self.Mb_neg)  # kN-m
        
        self.Ra_neg = self.R_calculator(self.Ma_neg, self.da)  # Mpa
        self.Ra_pos = self.R_calculator(self.Ma_pos, self.da)
        self.Ra_sup = self.R_calculator(self.Ma_sup, self.da)
        self.Rb_neg = self.R_calculator(self.Mb_neg, self.db)
        self.Rb_pos = self.R_calculator(self.Mb_pos, self.db)
        self.Rb_sup = self.R_calculator(self.Mb_sup, self.db)

        self.beta = self.BetaOne(fc)
        self.p_temp = self.reinforcement_ratio_temp(fy)

        self.As_a_neg = self.As_calculator(self.Ra_neg, self.da,fc,fy,self.beta)
        self.As_a_pos = self.As_calculator(self.Ra_pos, self.da,fc,fy,self.beta)
        self.As_a_sup = self.As_calculator(self.Ra_sup, self.da,fc,fy,self.beta)
        self.As_b_neg = self.As_calculator(self.Rb_neg, self.db,fc,fy,self.beta)
        self.As_b_pos = self.As_calculator(self.Rb_pos, self.db,fc,fy,self.beta)
        self.As_b_sup = self.As_calculator(self.Rb_sup, self.db,fc,fy,self.beta)

        self.S_a_neg = self.spacing(self.As_a_neg, self.bar_area, self.t)
        self.S_a_pos = self.spacing(self.As_a_pos, self.bar_area, self.t)
        self.S_a_sup = self.spacing(self.As_a_sup, self.bar_area, self.t)
        self.S_b_neg = self.spacing(self.As_b_neg, self.bar_area, self.t)
        self.S_b_pos = self.spacing(self.As_b_pos, self.bar_area, self.t)
        self.S_b_sup = self.spacing(self.As_b_sup, self.bar_area, self.t)
    
    def bar_area(self, d):
        return (math.pi / 4) * d**2
    
    def live_load_red(self, L, S, Ln, Sn, o_ll, ll_red):
        if ll_red == 'Yes' and L * S > 15 * 1000**2:
            R = 0.08 * (L * S * (1 / 1000)**2 - 15)
            return o_ll * (1 - R)
        return o_ll

    def thickness(self, L, S, fc, fy, wc):
        t = 2 * (S + L) / 180
        if isinstance(wc, int) and 1440 <= wc <= 1840 or wc == 'Normal Weight':
            t *= max(1.65 - 0.0003 * int(wc), 1.09) if isinstance(wc, int) else max(1.65 - 0.0003 * 1440, 1.09)
        if fy < 414:
            t *= (0.4 + (fy / 700))
        return max(100, min(math.ceil(t / 25) * 25, 250))

    def effective_depth_a(self, t, bar_dia):
        return t - 20 - bar_dia / 2

    def effective_depth_b(self, t, bar_dia):
        return t - 20 - bar_dia - bar_dia / 2

    def total_dl(self, o_dl, t, sw, include_sw):
        if include_sw == 'Yes':
            return o_dl + sw * (t / 1000)
        else:
            return o_dl

    def ultimate_load(self, o_dl, o_ll):
        return max(1.2 * o_dl + 1.6 * o_ll, 1.4 * o_dl)

    def load_dataframe(self, file_path):
        try:
            df = pd.read_csv(file_path)
            df.set_index('Ratio\\Case', inplace=True)
            df.index = df.index.astype(float)
            return df
        except FileNotFoundError:
            print(f"Error: The file {file_path} was not found.")
            return pd.DataFrame()
        except Exception as e:
            print(f"An error occurred while loading {file_path}: {e}")
            return pd.DataFrame()

    def calculate_aci_coefficients(self, Ln, Sn, c_case):
        m = Sn / Ln
        m_plus = round(math.ceil(m / 0.05) * 0.05, 2)
        m_minus = round(math.floor(m / 0.05) * 0.05, 2)

        case_column = f'{c_case}'

        result = {}

        def get_value(df, index, column):
            try:
                return df.at[index, column]
            except KeyError:
                print(f"KeyError: {index} not found in {column}")
                return 0

        if m == 1.0:
            result['ca_neg'] = get_value(self.ca_neg_df, 1.0, case_column)
            result['ca_posll'] = get_value(self.ca_posll_df, 1.0, case_column)
            result['ca_posdl'] = get_value(self.ca_posdl_df, 1.0, case_column)
            result['cb_neg'] = get_value(self.cb_neg_df, 1.0, case_column)
            result['cb_posll'] = get_value(self.cb_posll_df, 1.0, case_column)
            result['cb_posdl'] = get_value(self.cb_posdl_df, 1.0, case_column)
        elif m <= 0.5:
            result['ca_neg'] = get_value(self.ca_neg_df, 0.5, case_column)
            result['ca_posll'] = get_value(self.ca_posll_df, 0.5, case_column)
            result['ca_posdl'] = get_value(self.ca_posdl_df, 0.5, case_column)
            result['cb_neg'] = get_value(self.cb_neg_df, 0.5, case_column)
            result['cb_posll'] = get_value(self.cb_posll_df, 0.5, case_column)
            result['cb_posdl'] = get_value(self.cb_posdl_df, 0.5, case_column)
        else:
            if m_plus == m_minus:
                result['ca_neg'] = get_value(self.ca_neg_df, m, case_column)
                result['ca_posll'] = get_value(self.ca_posll_df, m, case_column)
                result['ca_posdl'] = get_value(self.ca_posdl_df, m, case_column)
                result['cb_neg'] = get_value(self.cb_neg_df, m, case_column)
                result['cb_posll'] = get_value(self.cb_posll_df, m, case_column)
                result['cb_posdl'] = get_value(self.cb_posdl_df, m, case_column)
            else:
                # For Ca negative
                ca_neg_value_plus = get_value(self.ca_neg_df, m_plus, case_column)
                ca_neg_value_minus = get_value(self.ca_neg_df, m_minus, case_column)
                result['ca_neg'] = ca_neg_value_plus + ((m - m_plus) / (m_minus - m_plus)) * (ca_neg_value_minus - ca_neg_value_plus)

                # For Ca positive Live load
                ca_posll_value_plus = get_value(self.ca_posll_df, m_plus, case_column)
                ca_posll_value_minus = get_value(self.ca_posll_df, m_minus, case_column)
                result['ca_posll'] = ca_posll_value_plus + ((m - m_plus) / (m_minus - m_plus)) * (ca_posll_value_minus - ca_posll_value_plus)

                # For Ca positive Dead load
                ca_posdl_value_plus = get_value(self.ca_posdl_df, m_plus, case_column)
                ca_posdl_value_minus = get_value(self.ca_posdl_df, m_minus, case_column)
                result['ca_posdl'] = ca_posdl_value_plus + ((m - m_plus) / (m_minus - m_plus)) * (ca_posdl_value_minus - ca_posdl_value_plus)

                # For Cb negative
                cb_neg_value_plus = get_value(self.cb_neg_df, m_plus, case_column)
                cb_neg_value_minus = get_value(self.cb_neg_df, m_minus, case_column)
                result['cb_neg'] = cb_neg_value_plus + ((m - m_plus) / (m_minus - m_plus)) * (cb_neg_value_minus - cb_neg_value_plus)

                # For Cb positive Live load
                cb_posll_value_plus = get_value(self.cb_posll_df, m_plus, case_column)
                cb_posll_value_minus = get_value(self.cb_posll_df, m_minus, case_column)
                result['cb_posll'] = cb_posll_value_plus + ((m - m_plus) / (m_minus - m_plus)) * (cb_posll_value_minus - cb_posll_value_plus)

                # For Cb positive Dead load
                cb_posdl_value_plus = get_value(self.cb_posdl_df, m_plus, case_column)
                cb_posdl_value_minus = get_value(self.cb_posdl_df, m_minus, case_column)
                result['cb_posdl'] = cb_posdl_value_plus + ((m - m_plus) / (m_minus - m_plus)) * (cb_posdl_value_minus - cb_posdl_value_plus)

        return result
    
    # Placeholder functions for other calculations
    def Moment_negative(self, L, ult_load, coefficient):
        return ((L/1000)**2) * ult_load * coefficient

    def Moment_positive(self, L, o_dl, o_ll, c_posll, c_posdl):
        return max((1.2*c_posdl*o_dl+1.6*c_posll*o_ll)*(L/1000)**2,1.4*c_posdl*o_dl*(L/1000)**2)

    def Moment_support(self, M_neg):
        return M_neg / 2
    
    def R_calculator(self,Mu,d):
        return (Mu*10**6) / (0.9*1000*d**2)

    def BetaOne(self, fc):
        if 17 <= fc <= 28:
            return 0.85
        if 28 < fc < 55:
            return 0.85 - 0.05 * (fc - 28) / 7
        if fc >= 55:
            return 0.65
        raise ValueError("fc must be greater than or equal to 17 MPa")

    def reinforcement_ratio_temp(self, fy):
        if fy < 420:
            return 0.002
        return max((0.0018 * 420) / fy, 0.0014)

    def As_calculator(self,R,d,fc,fy,beta):
        p_min = max((math.sqrt(fc))/(4*fy),1.4/fy)
        p_max = (3/8)*(0.85*fc*beta)/(fy)
        p_req = ((0.85*fc)/fy)*(1-math.sqrt(1-(2*R)/(0.85*fc)))

        if p_min < p_req and p_req < p_max:
            p = max(p_req,self.p_temp)
        elif p_req > p_max:
            raise ValueError('Increase Slab Thickness, to ensure ductile failure!')
        elif p_req < p_min:
            p = max(min((4/3)*p_req,p_min),self.p_temp)
        As = p*1000*d 
        return As

    def spacing(self,As,Ab,t):
        s = math.floor(min((1000*Ab)/As,3*t,450)/25)*25
        return s 

# Inputs
L = 4200     # Span length (meters)
S = 3175     # Span width (meters)
Ln = 3950    # Clear span length (meters)
Sn = 2925    # Clear span width (meters)
o_dl = 6.85  # Dead load (kN/m²)
o_ll = 2.4  # Live load (kN/m²)
bar_dia = 10  # Bar diameter (mm)
fc = 21     # Concrete compressive strength (MPa)
fy = 275    # Steel yield strength (MPa)
ll_red = 'Yes'  # Live load reduction (Yes/No)
wc = 'Normal Weight'  # Concrete unit weight
sw = 2.4    # Concrete specific weight (kN/m³)
c_case = '2'     # Load case identifier, corresponds to columns in the CSV files
include_sw = 'No'
# Initialize the TwoWaySlab object
slab = TwoWaySlab(L, S, Ln, Sn, o_dl, o_ll, bar_dia, fc, fy, ll_red, wc, sw, c_case, include_sw)

# Accessing and printing some calculated values
print(f"Short Span (mm): {slab.S}")
print(f"Short Span (mm): {slab.L}")
print(f"Slab thickness (mm): {slab.t}")
print(f"Short Clear Span (mm): {slab.Sn * 1000}")
print(f"Long Clear Span (mm): {slab.Ln * 1000}")
print(f"Ultimate load (kN/m²): {slab.ult_load}")
print(f"Dead Load (kN/m²): {slab.o_dl}")
print(f"Live Load (kN/m²): {slab.o_ll}")
print(f"Effective depth a (mm): {slab.da}")
print(f"Effective depth b (mm): {slab.db}")
print(f"ACI coefficients: {slab.aci_coefficients}")

print(f"Moment negative (Ma_neg) (kN-m): {slab.Ma_neg}")
print(f"Moment positive (Ma_pos) (kN-m): {slab.Ma_pos}")
print(f"Moment negative (Mb_neg) (kN-m): {slab.Mb_neg}")
print(f"Moment positive (Mb_pos) (kN-m): {slab.Mb_pos}")
print(f"Support moment Ma (kN-m): {slab.Ma_sup}")
print(f"Support moment Mb (kN-m): {slab.Mb_sup}")

print(f"Required reinforcement area As_a_neg (mm^2): {slab.As_a_neg}")
print(f"Required reinforcement area As_a_pos (mm^2): {slab.As_a_pos}")
print(f"Required reinforcement area As_a_sup (mm^2): {slab.As_a_sup}")
print(f"Required reinforcement area As_b_neg (mm^2): {slab.As_b_neg}")
print(f"Required reinforcement area As_b_pos (mm^2): {slab.As_b_pos}")
print(f"Required reinforcement area As_b_sup (mm^2): {slab.As_b_sup}")

print(f"Spacing for As_a_neg (mm): {slab.S_a_neg}")
print(f"Spacing for As_a_pos (mm): {slab.S_a_pos}")
print(f"Spacing for As_a_sup (mm): {slab.S_a_sup}")
print(f"Spacing for As_b_neg (mm): {slab.S_b_neg}")
print(f"Spacing for As_b_pos (mm): {slab.S_b_pos}")
print(f"Spacing for As_b_sup (mm): {slab.S_b_sup}")

Short Span (mm): 3175
Short Span (mm): 4200
Slab thickness (mm): 100
Short Clear Span (mm): 2925.0
Long Clear Span (mm): 3950.0
Ultimate load (kN/m²): 12.059999999999999
Dead Load (kN/m²): 6.85
Live Load (kN/m²): 2.4
Effective depth a (mm): 75.0
Effective depth b (mm): 65.0
ACI coefficients: {'ca_neg': 0.0699493670886076, 'ca_posll': 0.04575949367088608, 'ca_posdl': 0.02837974683544304, 'cb_neg': 0.0210506329113924, 'cb_posll': 0.01362025316455696, 'cb_posdl': 0.00862025316455696}
Moment negative (Ma_neg) (kN-m): 7.217434278797469
Moment positive (Ma_pos) (kN-m): 3.4992332971518985
Moment negative (Mb_neg) (kN-m): 3.9610165499999987
Moment positive (Mb_pos) (kN-m): 1.9216078499999996
Support moment Ma (kN-m): 3.6087171393987343
Support moment Mb (kN-m): 1.9805082749999994
Required reinforcement area As_a_neg (mm^2): 405.72501751216106
Required reinforcement area As_a_pos (mm^2): 256.4124162843881
Required reinforcement area As_a_sup (mm^2): 264.6053960838394
Required reinforcement area